In [1]:
%%bash
pip install numpy-quaternion

In [165]:
import numpy as np
import quaternion
import matplotlib.pyplot as plt

from scipy import optimize

from camera_calibration_parsers import readCalibration
import image_geometry

from tf import transformations

import time
from copy import deepcopy

import cv2

import rospy
import tf
from geometry_msgs.msg import PoseStamped, Pose, Vector3, Quaternion
from std_msgs.msg import Header

from IPython.display import clear_output

In [3]:
from client_mk1.fast_image import Image
from client_mk1.color import Color
from client_mk1.feat_2d import Circle2D
from client_mk1.searchers import do_linear_search, do_binary_search

In [4]:
%load_ext line_profiler
%load_ext autoreload
%autoreload 2
%aimport -rospy

In [5]:
rospy.init_node('fast_localization_client')
try:
    pose_pub.unregister()
except:
    pass
pose_pub = rospy.Publisher('/pose', PoseStamped, queue_size=10)

# Useful resources

[OpenCV tutorial](http://docs.opencv.org/3.0-beta/doc/py_tutorials/py_gui/py_video_display/py_video_display.html#playing-video-from-file)

![](http://wiki.ros.org/image_pipeline/CameraInfo?action=AttachFile&do=get&target=CameraCoords.png)

## Load camera configuration info

In [40]:
_, calibration = readCalibration("../calibration/unstabilized_pixel.yaml")
camera = image_geometry.PinholeCameraModel()
camera.fromCameraInfo(calibration)

# Define types for 3D features

In [152]:
class Sphere3D(object):
    """
    Sphere3D represents a sphere known to exist in 3D map-space.
    """
    def __init__(self, center, radius, colorRange):
        self.center = np.array(center)
        self.radius = radius
        self.color = colorRange

        # TODO: this should be a scalar with more reasonable behavior
        self.confidence = True
        
    def project(self, pos, orientation):
        """
        Returns the coordinates of the given point in the camera image
        """
        relative_center = self.center - pos
        relative_center = np.dot(np.transpose(orientation), relative_center)
        
        center2D = camera.project3dToPixel(relative_center)
        x, y, w = relative_center
        radius2D = camera.fx() * self.radius / w
        
        return Circle2D(center2D, radius2D, self.color)


In [153]:
s = greenSphere
s.project(startPos,np.identity(3))

Circle(center=[ 1255.15801843   766.60709505], radius=102.317940611)

# Define hardcoded map for testing tracking

In [154]:
green = Color(minimum=(30, 50, 100), maximum=(60, 255, 255))
orange = Color(minimum=(5, 120, 130), maximum=(30, 255, 255))
blue = Color(minimum=(90, 60, 90), maximum=(150, 255, 255))
black = Color(minimum=(0, 0, 0), maximum=(255, 210, 150))

# greenCircle = Circle2D((1920/2, 1080/2), 400, green)
# orangeCircle = Circle2D((1920/2, 1080/2), 400, orange)

greenSmall = Circle2D((1100, 700), 130, green)
orangeSmall = Circle2D((600, 700), 130, orange)

greenSphere = Sphere3D((.55/2,.45/2,0), .106, green)
orangeSphere = Sphere3D((-.55/2,.45/2,0), .106, orange)
blueSphere = Sphere3D((.55/2,-.45/2,0), .106, blue)
blackSphere = Sphere3D((-.55/2,-.45/2,0), .106, black)


startPos = np.array([0, 0, -1.5])
startQuat = np.quaternion(1, 0, 0 ,0)


greenCircle = greenSphere.project(startPos, quaternion.as_rotation_matrix(startQuat))
orangeCircle = orangeSphere.project(startPos, quaternion.as_rotation_matrix(startQuat))
blueCircle = blueSphere.project(startPos, quaternion.as_rotation_matrix(startQuat))
blackCircle = blackSphere.project(startPos, quaternion.as_rotation_matrix(startQuat))

# Find 3D poses

In [155]:
def packStateVector(pos, quat):
    return np.array(tuple(pos)+tuple(quaternion.as_float_array(quat).ravel()))

def unpackStateVector(state):
    position = np.array(state[:3])
    quat = np.quaternion(*state[3:])
    return position, quat

In [156]:
packStateVector([-0.07634509, -0.01887565, -2.86295623],np.quaternion(0.999615505058199, -0.0100498077476995, 0.0505565336078942, -0.00213947738153386))

array([ -7.63450900e-02,  -1.88756500e-02,  -2.86295623e+00,
         9.99615505e-01,  -1.00498077e-02,   5.05565336e-02,
        -2.13947738e-03])

In [157]:

def getDistFunc(spheres, foundCircles):
    assert len(spheres) == len(foundCircles)
    def distFunc(state):
        position, quat = unpackStateVector(state)
        """
        Takes a list of 7 numbers. The first three represent translational 
        position, and the last four form a rotation quaternion.
        Returns a list of error values.
        """
        matrix = quaternion.as_rotation_matrix(quat)
        
        errors = [c.distance(s.project(position, matrix).center)
         for s, c in zip(spheres, foundCircles)
                 if c.confidence == True]
        
        # Flatten the resulting 2-deep list
        errors = [e for l in errors for e in l]
        
        # Note that scipy leastsq requires sufficient number of outputs,
        # hacked here by feeding extra '0's
        return errors + [quat.norm() - 1] + [0]*6
        
        
    return distFunc

In [158]:
quat = np.quaternion(1,2,3,4)

In [159]:
getDistFunc([greenSphere, orangeSphere], [greenSmall, orangeSmall]
           )(packStateVector(startPos, startQuat))

[155.15801843333338,
 66.607095050000112,
 124.26304356666662,
 66.607095050000112,
 0.0,
 0,
 0,
 0,
 0,
 0,
 0]

In [160]:
# TODO: massive speed boosts should come from computing
# the Jacobian more sensibly
def findPose(spheres, circles, pos, quat, verbose=False):
    p,cov,infodict,mesg,ier = optimize.leastsq(
        getDistFunc(spheres, circles),
        packStateVector(pos, quat),
        xtol=5e-4,
        full_output=1)
    if verbose:
        print '{} evaluations needed, final error {}'.format(infodict['nfev'], infodict['fvec'])
    return unpackStateVector(p)

In [161]:
findPose([greenSphere, orangeSphere], [greenSmall, orangeSmall],
         startPos, startQuat, verbose=True)

25 evaluations needed, final error [  5.20966955e-06   6.28343139e-06  -7.97606367e-06   4.32512263e-06
   2.09201323e-10   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00]


(array([ 0.        ,  0.        , -1.59987096]),
 quaternion(0.998757418295752, -0.0179969907785118, 0.0461291306120982, -0.00564191813975188))

# ROS functionality

In [257]:
def publish_pose(pos, quat):
    # Convert the position and quaternion from OpenCV coordinates to ROS coordinates
    pos = [pos[2], -pos[0], -pos[1]]
    
    quat = np.quaternion(0,0,0,1) * quat
    quatList = quaternion.as_float_array(quat.normalized()).flatten()
    quatList[1], quatList[2] = quatList[2], quatList[1]
    
    quatList[2] *= -1
    quatList[3] *= -1
    
    # Pulish the result as a Pose in map frame
    pose_pub.publish(PoseStamped(
        header=Header(frame_id='map'),
        pose=Pose(
            position=Vector3(*pos), 
            orientation=Quaternion(*quatList))))
    
    # Publish the result as a transform from map frame to camera frame
    br = tf.TransformBroadcaster()
    br.sendTransform(pos,
                     quatList,
                     rospy.Time.now(),
                     'camera',
                     'map')


# Capture and handle video

In [258]:
FILE = '../test_data/unstabilized_cardinal.mp4'
cv2.namedWindow('raw_frame')

In [259]:
cap = cv2.VideoCapture(FILE)

def handleFrame(frame, features, verbose=False):
    image = Image(frame)

    for f in features:
        f.refine(image, verbose=verbose)
    
    return image


def main():
    pos = startPos
    quat = startQuat
#     pos = np.array([-1, 0, -1])
#     quat = np.quaternion(1,0,.5,0).normalized()
    spheres = deepcopy([greenSphere, orangeSphere, blueSphere, blackSphere])
    i=0
    while cap.isOpened():
        verbose = i%50 == 0
        if verbose:
            clear_output()
        i += 1
        
        startTime = time.time()
        ret, frame = cap.read()
        readTime = time.time()

        if frame is None or len(frame) <= 0:
            print "End of video file reached"
            break

        # Do work here
        features = [sphere.project(pos, quaternion.as_rotation_matrix(quat))
                   for sphere in spheres]
        startFeats = deepcopy(features)
        
        image = handleFrame(frame, features, verbose=verbose)
        
        pos, quat = findPose(spheres, features, pos, quat, verbose=verbose)
#         quat = np.quaternion(1,0,0,0)
            
        finishTime = time.time()
        
        publish_pose(pos, quat)

        for f in features:
            f.draw(frame)
            
        image.draw(frame)
        scaled = cv2.resize(frame, dsize=None, fx=0.5, fy=0.5)
        cv2.imshow('raw_frame', scaled)
        
        drawnTime = time.time()

        cv2.waitKey(1)
        
        if verbose:
            processingTime = (finishTime-readTime)
            print "{0:.1f}ms reading and drawing image, {1:.1f}ms ({3}fps) processing, {2} pixels accessed\n".format(
                (readTime-startTime + drawnTime-finishTime)*1000, 
                processingTime*1000, 
                len(image.pixelsAccessed),
                int(1/processingTime))
        
main()

41 evaluations needed, final error [  1.19819547e+00   8.16461403e-01   8.01581127e-01  -1.73493383e-01
  -1.69383093e+00   7.89161565e-01  -2.62837912e-01  -1.45885583e+00
   2.10406327e-06   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00]
33.8ms reading and drawing image, 35.2ms (28fps) processing, 148 pixels accessed

End of video file reached


# Profile things

In [35]:
cap = cv2.VideoCapture(FILE)

def bench():
    features = deepcopy([greenSmall])
    for _ in range(60):
        ret, frame = cap.read()
        handleFrame(frame, features)
    
%lprun -r -f do_binary_search bench()